In [1]:
import sys
sys.path.append('../')
from deep_rl import *
from deep_rl.agent.DSFPG_agent import DSFPGAgent
from deep_rl.agent.DSFPG_delayed import DSFPGAgent_td3

import sys
sys.path.append('../')
from deep_rl import *
from deep_rl.agent.DSFPG_agent import DSFPGAgent

select_device(0)

No module named 'flow'
No module named 'carla'


In [3]:
def dsfpg_online(**kwargs):
    generate_tag(kwargs)
    kwargs.setdefault('log_level', 0)
    '''kwargs.setdefault('n_step', 1)
    kwargs.setdefault('replay_cls', UniformReplay)
    kwargs.setdefault('async_replay', True)'''
    
    config = Config()
    config.merge(kwargs)

    config.offline = False
    config.task_fn = lambda: Task(config.game)
    config.eval_env = config.task_fn()
    config.max_steps = int(1e6)
    config.eval_interval = int(1e4)
    config.eval_episodes = 10
    config.td3_delay = 2
    
    config.network_fn = lambda: DSFPGNet(
        config.state_dim, config.action_dim,
        actor_body=FCBody(config.state_dim, (400, 300), gate=F.relu),
        # critic_body=FCBody(config.state_dim, (400, 300), gate=F.relu),
        critic_body=SF_FCBody(config.state_dim, (400, 300), gate=F.relu, linear=config.linear),
        sf_body=FCBody(config.state_dim+config.action_dim, (400, 300), gate=F.relu),
        actor_opt_fn=lambda params: torch.optim.Adam(params, lr=1e-3),
        critic_opt_fn=lambda params: torch.optim.Adam(params, lr=1e-3),
        sf_opt_fn=lambda params: torch.optim.Adam(params, lr=1e-3))

    # config.replay_fn = lambda: ReplayWrapper(UniformReplay, replay_kwargs)

    config.replay_fn = lambda: UniformReplay(memory_size=int(1e6), batch_size=256)
    config.discount = 0.99
    config.random_process_fn = lambda: OrnsteinUhlenbeckProcess(
        size=(config.action_dim,), std=LinearSchedule(0.2))
    ''' config.random_process_fn = lambda: GaussianProcess(
        size=(config.action_dim,), std=LinearSchedule(0.1))'''
    config.warm_up = int(1e4)
    # config.pre_training_steps = int(1e6)
    config.target_network_mix = 5e-3
    # run_steps(DSFPGAgent(config))
    
    '''
    change the following line to switch between vanilla dsfpg / delayed vanilla dsfpg
    '''
    agent = DSFPGAgent_td3(config)
    # config = agent.config
    agent_name = agent.__class__.__name__
    t0 = time.time()
    agent.pre_training = False
    '''while True:
        if config.log_interval and not agent.unsupervised_steps % config.log_interval:
            agent.logger.info('steps %d, %.2f steps/s' % (agent.unsupervised_steps, config.log_interval / (time.time() - t0)))
            t0 = time.time()
        if config.max_steps and agent.unsupervised_steps >= config.pre_training_steps:
            agent.close()
            break
        agent.step()
        agent.switch_task()'''


    print('==================== Start RL Phase =====================')
    agent.pre_training = False
    while True:
        if config.save_interval and not agent.total_steps % config.save_interval:
            agent.save('data/%s-%s-%d' % (agent_name, config.tag, agent.total_steps))
        if config.log_interval and not agent.total_steps % config.log_interval:
            agent.logger.info('steps %d, %.2f steps/s' % (agent.total_steps, config.log_interval / (time.time() - t0)))
            t0 = time.time()
        if config.eval_interval and not agent.total_steps % config.eval_interval:
            agent.eval_episodes()
        if config.max_steps and agent.total_steps >= config.max_steps:
            return agent
            break
        agent.step()
        agent.switch_task()
    




dsfpg_linear = dsfpg_online(game='Hopper-v2', dataset_name='hopper-expert-v0', linear=True)

2021-06-28 22:01:58,865 - root - INFO: steps 0, 85598040.82 steps/s


False
==================== Start RL Phase =====================


2021-06-28 22:01:59,413 - root - INFO: steps 0, episodic_return_test 158.86(16.58)
2021-06-28 22:01:59,418 - root - INFO: steps 15, episodic_return_train 10.347426300384845
2021-06-28 22:01:59,421 - root - INFO: steps 28, episodic_return_train 8.578600713183567
2021-06-28 22:01:59,428 - root - INFO: steps 59, episodic_return_train 30.350026153300142
2021-06-28 22:01:59,432 - root - INFO: steps 81, episodic_return_train 25.05855221218947
2021-06-28 22:01:59,439 - root - INFO: steps 115, episodic_return_train 33.49199615391716
2021-06-28 22:01:59,441 - root - INFO: steps 126, episodic_return_train 7.552477707375726
2021-06-28 22:01:59,446 - root - INFO: steps 145, episodic_return_train 11.389067704426747
2021-06-28 22:01:59,451 - root - INFO: steps 161, episodic_return_train 9.9121882054308
2021-06-28 22:01:59,453 - root - INFO: steps 173, episodic_return_train 10.665809925812857
2021-06-28 22:01:59,459 - root - INFO: steps 200, episodic_return_train 13.356832462269166
2021-06-28 22:01:5

In [4]:
dsfpg_nn = dsfpg_online(game='Hopper-v2', dataset_name='hopper-expert-v0', linear=False)

2021-06-29 00:08:54,229 - root - INFO: steps 0, 116508444.44 steps/s


False
==================== Start RL Phase =====================


2021-06-29 00:08:54,743 - root - INFO: steps 0, episodic_return_test 142.54(5.19)
2021-06-29 00:08:54,748 - root - INFO: steps 15, episodic_return_train 9.783773799533641
2021-06-29 00:08:54,752 - root - INFO: steps 33, episodic_return_train 15.546094216363713
2021-06-29 00:08:54,758 - root - INFO: steps 60, episodic_return_train 11.166649746109918
2021-06-29 00:08:54,761 - root - INFO: steps 75, episodic_return_train 11.512062491482888
2021-06-29 00:08:54,765 - root - INFO: steps 91, episodic_return_train 7.531402074216892
2021-06-29 00:08:54,767 - root - INFO: steps 100, episodic_return_train 6.2335356749629085
2021-06-29 00:08:54,770 - root - INFO: steps 116, episodic_return_train 8.902992811731094
2021-06-29 00:08:54,776 - root - INFO: steps 150, episodic_return_train 43.80663334811711
2021-06-29 00:08:54,785 - root - INFO: steps 195, episodic_return_train 47.1290871220081
2021-06-29 00:08:54,787 - root - INFO: steps 206, episodic_return_train 8.219931638870765
2021-06-29 00:08:54,

KeyboardInterrupt: 

In [5]:
def ddpg_continuous(**kwargs):
    generate_tag(kwargs)
    kwargs.setdefault('log_level', 0)
    config = Config()
    config.merge(kwargs)

    config.task_fn = lambda: Task(config.game)
    config.eval_env = config.task_fn()
    config.max_steps = int(1e6)
    config.eval_interval = int(1e4)
    config.eval_episodes = 20

    config.network_fn = lambda: DeterministicActorCriticNet(
        config.state_dim, config.action_dim,
        actor_body=FCBody(config.state_dim, (400, 300), gate=F.relu),
        critic_body=FCBody(config.state_dim + config.action_dim, (400, 300), gate=F.relu),
        actor_opt_fn=lambda params: torch.optim.Adam(params, lr=1e-3),
        critic_opt_fn=lambda params: torch.optim.Adam(params, lr=1e-3))

    config.replay_fn = lambda: UniformReplay(memory_size=int(1e6), batch_size=256)
    config.discount = 0.99
    config.random_process_fn = lambda: OrnsteinUhlenbeckProcess(
        size=(config.action_dim,), std=LinearSchedule(0.2))
    config.warm_up = int(1e4)
    config.target_network_mix = 5e-3
    run_steps(DDPGAgent(config))

    
ddpg = ddpg_continuous(game='Hopper-v2', dataset_name='hopper-expert-v0')

2021-06-29 02:02:37,836 - root - INFO: steps 0, 419430400.00 steps/s
2021-06-29 02:02:38,889 - root - INFO: steps 0, episodic_return_test 149.93(6.37)
2021-06-29 02:02:38,893 - root - INFO: steps 11, episodic_return_train 8.122180348057189
2021-06-29 02:02:38,899 - root - INFO: steps 40, episodic_return_train 31.034261059405043
2021-06-29 02:02:38,901 - root - INFO: steps 53, episodic_return_train 8.327916414675197
2021-06-29 02:02:38,905 - root - INFO: steps 71, episodic_return_train 12.505988092019253
2021-06-29 02:02:38,912 - root - INFO: steps 101, episodic_return_train 21.756289397538573
2021-06-29 02:02:38,914 - root - INFO: steps 112, episodic_return_train 5.322004604292696
2021-06-29 02:02:38,919 - root - INFO: steps 131, episodic_return_train 18.845680668182688
2021-06-29 02:02:38,923 - root - INFO: steps 154, episodic_return_train 10.787765009359585
2021-06-29 02:02:38,927 - root - INFO: steps 176, episodic_return_train 25.005587994451332
2021-06-29 02:02:38,936 - root - INFO